# 测试
1. 同步参数
2. 提取1次梯度
3. 独立learner应用后对比算法迭代后模型参数是否一致

In [1]:
from ray.rllib.algorithms.ppo import PPOConfig
import sys
sys.path.append(r"..")
from easy_helper import simplify_rllib_metrics
from ray.rllib.algorithms.ppo.torch.ppo_torch_learner import PPOTorchLearner
from ray.rllib.core import COMPONENT_RL_MODULE
import copy

client_gradients = None
client_params = None

def get_weights(learner):
    return learner.get_state(components=COMPONENT_RL_MODULE)['rl_module']['default_policy']

class DebugLearner(PPOTorchLearner):
    def apply_gradients(self, gradients_dict) -> None:
        res = super().apply_gradients(gradients_dict)
        global client_params
        if client_params is None:
            print("save client_params")
            client_params = get_weights(self)
        return res  
    
    def compute_gradients(self, loss_per_module, **kwargs):
        res = super().compute_gradients(loss_per_module, **kwargs)
        global client_gradients
        if client_gradients is None:
            print("save client_gradients")
            client_gradients = copy.deepcopy(res)
        return res

config = (
    PPOConfig()
    .api_stack(
        enable_rl_module_and_learner=True,
        enable_env_runner_and_connector_v2=True,
    )
    .api_stack(
        enable_rl_module_and_learner=True,
        enable_env_runner_and_connector_v2=True,
    )
    .environment("CartPole-v1")
    .env_runners(num_env_runners=0)
    .training(
        learner_class=DebugLearner,# 调试学习者
    )
)

algo = config.build()
init_params = algo.learner_group.get_weights()
result = algo.train()

2025-01-18 16:45:55,042	WARNING ppo.py:295 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
d:\programs\miniconda3\Lib\site-packages\ray\rllib\algorithms\algorithm.py:569: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
d:\programs\miniconda3\Lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ig

save client_gradients
save client_params


2025-01-18 16:46:21,159	INFO worker.py:1821 -- Started a local Ray instance.


In [2]:
client_gradients

{Parameter containing:
 tensor([[-0.4766, -0.0797, -0.0656,  0.2410],
         [ 0.1807,  0.4328, -0.2064,  0.1337],
         [-0.2560, -0.4437, -0.3879,  0.0840],
         ...,
         [ 0.4761, -0.0789, -0.1685,  0.0944],
         [-0.2891,  0.0593, -0.3033,  0.0763],
         [-0.4968, -0.1642,  0.1161,  0.3846]], requires_grad=True): tensor([[ 9.0610e-06,  7.7925e-05, -1.0062e-05, -1.3299e-04],
         [ 1.4983e-06,  1.2714e-05, -4.5003e-06, -2.2837e-05],
         [-4.0701e-06, -3.6643e-05,  4.3349e-06,  6.2296e-05],
         ...,
         [ 5.2412e-06,  4.1994e-05, -4.5706e-06, -7.2516e-05],
         [ 4.2949e-06,  4.0159e-05,  1.5197e-06, -6.4364e-05],
         [-2.8493e-06, -2.4512e-05,  3.8938e-06,  4.2564e-05]]),
 Parameter containing:
 tensor([ 0.1054,  0.2365, -0.1138,  0.3150, -0.2349, -0.1317,  0.2437,  0.0813,
          0.1033,  0.4258, -0.1194,  0.2498, -0.3351,  0.0372,  0.2235, -0.3241,
          0.3867,  0.3347, -0.0207,  0.2646, -0.2914, -0.0013, -0.4835, -0.0971,


In [3]:
client_params

OrderedDict([('encoder.actor_encoder.net.mlp.0.weight',
              array([[-0.47664642, -0.07977663, -0.06555974,  0.24102621],
                     [ 0.18060105,  0.43276006, -0.20632267,  0.13376513],
                     [-0.25592548, -0.44360217, -0.38794994,  0.08394761],
                     ...,
                     [ 0.476077  , -0.07890115, -0.16842479,  0.09447615],
                     [-0.28918195,  0.05921652, -0.30336425,  0.07636265],
                     [-0.4967575 , -0.16415116,  0.1160868 ,  0.3845624 ]],
                    dtype=float32)),
             ('encoder.actor_encoder.net.mlp.0.bias',
              array([ 0.10538603,  0.2365719 , -0.11371931,  0.315037  , -0.23485161,
                     -0.13170622,  0.24379417,  0.08138584,  0.10336994,  0.42577648,
                     -0.11945914,  0.2497216 , -0.33510926,  0.03714253,  0.22357313,
                     -0.32412654,  0.38673645,  0.3346965 , -0.02061255,  0.26456964,
                     -0.29149318

In [4]:
import gymnasium as gym
env = gym.make("CartPole-v1")

learner = config.build_learner(env=env)
# Needs to be called on the learner before calling any functions.
learner.build()
weights = {COMPONENT_RL_MODULE: init_params}
learner.set_state(weights)


2025-01-18 16:46:24,198	WARNING rl_module.py:427 -- Could not create a Catalog object for your RLModule! If you are not using the new API stack yet, make sure to switch it off in your config: `config.api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)`. Some algos already use the new stack by default. Ignore this message, if your RLModule does not use a Catalog to build its sub-components.


In [5]:
def apply_gradients(gradients_dict, learner):
    """更新参数"""
    params = learner._params
    for k, v in zip(list(params.keys()), list(gradients_dict.values())):
        params[k].grad = v.to(learner._device)
    
    learner.apply_gradients({})
    return get_weights(learner)

server_params = apply_gradients(client_gradients, learner)

In [6]:
# 判断是否一致
import numpy as np
for idx, (k, _client), (_, _server) in zip(range(len(client_params)),  client_params.items(), server_params.items()):
    print(idx, np.array_equal(_client, _server))


0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True


: 